# LlaMA mit RAG

In [1]:
user_question = "Was geschieht in der Vorweihnachtszeit in der Geschichte 'Der kleine Schmied'?"

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
question_embedding = model.encode([user_question])

In [3]:
import json
import faiss
import numpy as np

# Mapping laden
with open("segment_mapping.json", "r", encoding="utf-8") as f:
    segment_mapping = json.load(f)

# Anzahl der relevanten Segmente, die abgerufen werden sollen
k = 40

# FAISS-Index laden
index = faiss.read_index("faiss_index.db")

# Suche durchführen
D, I = index.search(np.array(question_embedding), k)

# Relevante Texte anhand der IDs abrufen, -1 filtern
relevant_segments = [segment_mapping[str(i)] for i in I[0] if i != -1]

# Überprüfen, ob relevante Segmente gefunden wurden
if not relevant_segments:
    print("Keine relevanten Segmente gefunden.")

In [4]:
# Relevante Segmente zu einem Kontextstring zusammenfassen
context = " ".join(relevant_segments)

In [5]:
# API-Anfrage mit Kontext vorbereiten
api_request_json = {
    "messages": [
        {
            "role": "system",
            "content": (
                f"Hier ist zusätzlicher Kontext, falls du ihn für die Beantwortung von Fragen verwenden möchtest: {context}. "
                "Falls der Kontext nicht relevant ist, antworte basierend auf deinem allgemeinen Wissen."
            ),
        },
        {"role": "user", "content": user_question},
    ],
    "stream": False,
}

In [6]:
from llamaapi import LlamaAPI

llama = LlamaAPI("LA-f0461f58f1794f6f93072f5f112bd7a44ca2c90adb1a4842a8dea42acb05317f")

# Anfrage ausführen und Antwort verarbeiten
response = llama.run(api_request_json)

# Antwort als JSON extrahieren
response_json = response.json()

# Zugriff auf die Inhalte
content = response_json['choices'][0]['message']['content']

# Antwort anzeigen
print(f"Antwort: {content}")

Antwort: In der Vorweihnachtszeit in der Geschichte "Der kleine Schmied" werden die Landwirte ihre Gerätschaften zum Überprüfen bringen, bevor sie geschärft und gerichtet werden müssen.
